In [ ]:
import os

import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from trino.dbapi import connect
from dotenv import load_dotenv

load_dotenv()

conn = connect(
    host=os.getenv("TRINO_HOST", "localhost"),
    port=int(os.getenv("TRINO_PORT", 8080)),
    user=os.getenv("TRINO_USER", "trino")
)


In [ ]:
def trino_executor(conn, sql_file_path) -> pl.DataFrame:
    with open(sql_file_path, "r", encoding="utf-8") as f:
        sql = f.read()

    with conn.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]

    df = pl.DataFrame(rows, schema=columns, infer_schema_length=10000)
        
    return df

In [ ]:
df_before = trino_executor(conn, "./query.sql")
df_count = df_before = trino_executor(conn, "./query/bef.sql").group_by("ad_datetime_id").agg(pl.len()).rename({"len": "count"})

plt.figure(figsize=(20, 5))
sns.lineplot(x="ad_datetime_id", y="count", data=df_count)
plt.xlabel("ad_datetime_id")
plt.ylabel("count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
